In [2]:
import ciropt as co
import cvxpy as cp

In [3]:
L_smooth = 1.
mu = 0.0001
Capacitance = 1.

# solver = "ca_canonical_X"
solver = "gp_canonical_X"

time_limit = 100

# Ciropt problem

In [4]:
problem = co.gradient_flow_circuit(mu, L_smooth, Capacitance)

res, model, sp_exp = problem.solve(solver=solver, verbose=False, time_limit=100)[:3]
print(res)

dim_G=5, dim_F=4
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-29
<class 'numpy.ndarray'> <class 'gurobipy.MQuadExpr'> <class 'numpy.ndarray'>


c:\Users\owner\Documents\GitHub\optimization_via_circuits\examples\ciropt\circuit_opt.py:495: RuntimeWarning: Chained matrix multiplications of MVars is inefficient, collect numeric terms first when building expressions
  model.addConstr(  (M1 @ var_x).item()  - gp_trace(M2 @ var_X) == 0)


<class 'numpy.ndarray'> <class 'gurobipy.MQuadExpr'> <class 'numpy.ndarray'>
<class 'numpy.ndarray'> <class 'gurobipy.MQuadExpr'> <class 'numpy.ndarray'>
<class 'numpy.ndarray'> <class 'gurobipy.MQuadExpr'> <class 'numpy.ndarray'>


c:\Users\owner\Documents\GitHub\optimization_via_circuits\examples\ciropt\circuit_opt.py:509: RuntimeWarning: Chained matrix multiplications of MVars is inefficient, collect numeric terms first when building expressions
  + gp_trace(M2 @ var_X) \
c:\Users\owner\Documents\GitHub\optimization_via_circuits\examples\ciropt\circuit_opt.py:510: RuntimeWarning: Chained matrix multiplications of MVars is inefficient, collect numeric terms first when building expressions
  - gp_trace(M3 @ var_X) == 0)
c:\Users\owner\Documents\GitHub\optimization_via_circuits\examples\ciropt\circuit_opt.py:525: RuntimeWarning: Chained matrix multiplications of MVars is inefficient, collect numeric terms first when building expressions
  model.addConstr( gp_trace(M1 @ var_X) + (M2 @ var_x).item() == 0)


{'b': 0.9999999970141082, 'h': 2.33756135519335, 'd': 1000.0, 'alpha': 0.42778288607438625, 'beta': 0.4277463601699642}


In [5]:
print(res)

{'b': 0.9999999970141082, 'h': 2.33756135519335, 'd': 1000.0, 'alpha': 0.42778288607438625, 'beta': 0.4277463601699642}


In [6]:
res

{'b': 0.9999999970141082,
 'h': 2.33756135519335,
 'd': 1000.0,
 'alpha': 0.42778288607438625,
 'beta': 0.4277463601699642}

# PEP verification

In [7]:
import PEPit
from PEPit.functions import SmoothStronglyConvexFunction, SmoothConvexFunction
from PEPit.primitive_steps import proximal_step

b = res["b"] 
h = res["h"]
alpha = res["alpha"]
beta = res["beta"]

In [8]:
problem = PEPit.PEP()
func = problem.declare_function(
                    SmoothStronglyConvexFunction,
                    mu=mu,   # Strong convexity param.
                    L=L_smooth) 


x_P_star = func.stationary_point()
g_star, f_star = func.oracle(x_P_star)
x_P_1 = problem.set_initial_point()

g_1, f_1 = func.oracle(x_P_1)
x_P_1p5 = x_P_1 - (alpha * h / Capacitance) * g_1 
g_1p5, f_1p5 = func.oracle(x_P_1p5)
x_P_2 = x_P_1  - (beta * h / Capacitance) * g_1 - ((1 - beta) * h / Capacitance) * g_1p5

g_2, f_2 = func.oracle(x_P_2)

# Set the performance metric to the function values accuracy
E_1 = (Capacitance/2) * (x_P_1 - x_P_star)**2
E_2 = (Capacitance/2) * (x_P_2 - x_P_star)**2
Delta_1 = b * (f_1 - f_star)
problem.set_performance_metric(E_2 - (E_1 - Delta_1))

# Solve the PEP with verbose = 1
verbose = 0
diff = problem.solve(verbose=verbose, solver=cp.MOSEK)
print(f"{diff=}")

diff=-6.809414920816437e-09
